## Юнит 9. Профориентация в Data Science
### Skillfactory: DSPR-19
### Проект 7. Возьмёте Бэтмобиль? 

### 1. Чем займёмся в этом проекте?
→ Поздравляем! Вы приступаете к выполнению финального проекта блока Профориентация, в котором вы сможете на практике поработать с задачами из всех трёх направлений — ML, CV и NLP.

За основу возьмем решение, которое нам уже известно, — построенную на табличных данных модель «Предсказание стоимости автомобиля» — и добавим туда модели Deep Learning, которые будут распознавать изображения и обрабатывать текст.

За последние несколько месяцев работы в компании «Старый друг» вы разработали две модели, которые уже успешно крутятся в продакшне и приносят деньги компании и радость сотрудникам (по словам Анны из отдела продаж).

→ Осталось довести свою работу до логичного завершения и, обогатив датасет текстовыми данными из объявлений о продаже, свести все наши модели в единое решение — Multi-inputs сеть. Приступим?

### КАК БУДЕМ РЕШАТЬ ПРОЕКТ?

### ПОСТРОИМ БАЗОВОЕ РЕШЕНИЕ:

- Шаг 1. Построим «наивную» ML-модель на табличных данных, которая предсказывает цену по модели и году выпуска. Это позволит нам задать направление для дальнейших экспериментов. Впоследствии на этом этапе вы можете доработать вашу модель из проекта «Выбираем авто выгодно». 
- Шаг 2. Обработаем и отнормируем признаки и сделаем первую модель на основе градиентного бустинга с помощью CatBoost.
- Шаг 3. Решим эту же задачу с помощью DL (модель NN Tabular) и сравним результаты.
- Шаг 4. Добавим текстовые данные (NLP) и сделаем Multi-Input нейронную сеть для анализа и табличных данных, и текста одновременно.
- Шаг 5. Добавим обработку изображений в Multi-Input нейронную сеть.
- Шаг 6. Осуществим ансамблирование градиентного бустинга и нейронной сети (усредним их предсказания).

### САМОСТОЯТЕЛЬНАЯ ЧАСТЬ

- Ознакомьтесь с критериями оценивания проекта.
- Следуйте нашим рекомендациям, чтобы построить самую лучшую модель.
- Задавайте вопросы одногруппникам и менторам в Slack.
- Загрузите итоговое решение на kaggle и добавьте описание проекта на git.  

На выполнение проекта отводится: две недели.

### ЧТО ВЫ ПОЛУЧИТЕ В РЕЗУЛЬТАТЕ ПРОЕКТА?

- Потренируетесь обрабатывать данные и оптимизировать NLP-модели.
- Пройдетесь по всему циклу разработки комплексной Multi-Inputs модели (от обработки данных до внедрения в продакшн).
- Увидите, как использование blend-решений влияет на целевые метрики.
- Примерите на себя роль специалиста каждого из трёх треков второго года обучения.



— Возьмете Бэтмобиль?  
— Нет. Слишком заметно.  
— Ну тогда возьмите Ламборгини. Это менее заметно.

### 2. Начнём с простой модели

Начнём с разработки baseline нашего решения и посмотрим, как происходит обращение с входящими данными и что нужно получить на выходе. 

По ссылке вы обнаружите baseline. 

СКАЧАТЬ BASELINE (https://github.com/luhakv/cv_ml_nlp/blob/master/baseline-sf-dst-car-price-part2-v6.ipynb)

Итак, наша задача — предсказать стоимость автомобиля по его характеристикам, текстовому описанию или картинке. 

Начнём знакомиться с baseline-решением. Как видим, данные не сильно отличаются от прежних:

bodyType — категориальный  
brand —  категориальный   
color —  категориальный  
description —  текстовый  
engineDisplacement — числовой, представленный как текст  
enginePower — числовой, представленный как текст  
fuelType — категориальный  
mileage —  числовой   
modelDate — числовой  
model_info — категориальный  
name — категориальный, желательно сократить размерность  
numberOfDoors — категориальный  
price — числовой, целевой  
productionDate — числовой  
sell_id — изображение (файл доступен по адресу, основанному на sell_id)  
vehicleConfiguration — не используется (комбинация других столбцов)  
vehicleTransmission — категориальный  
Владельцы — категориальный  
Владение — числовой, представленный как текст  
ПТС — категориальный  
Привод — категориальный  
Руль — категориальный  


Однако кое-что, скорее всего, придётся переделать, поскольку есть дополнительные столбцы. 

→ Переходим к построению самой простой «наивной» модели, которая предсказывает среднюю цену по модели автомобиля и году выпуска. Начинать решение всегда стоит именно с простой модели, чтобы понимать, куда вы движетесь — к улучшению или к ухудшению решения. Это сэкономит вам множество сил и ресурсов. 

Наша модель предсказывает среднее значение по стоимости относительно модели и года выпуска.



In [ ]:
# Наивная модель
predicts = []
for index, row in pd.DataFrame(data_test[['model_info', 'productionDate']]).iterrows():
    query = f"model_info == '{row[0]}' and productionDate == '{row[1]}'"
    predicts.append(data_train.query(query)['price'].median())

# заполним не найденные совпадения
predicts = pd.DataFrame(predicts)
predicts = predicts.fillna(predicts.median())

# округлим
predicts = (predicts // 1000) * 1000

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(data_test['price'], predicts.values[:, 0]))*100:0.2f}%")

### 3. EDA и модель на основе градиентного бустинга

На этом этапе нам важно посмотреть на распределение числовых признаков, опуская категориальные, поскольку задача этого шага — посмотреть, подходят ли данные для загрузки в сеть. 




In [ ]:
#посмотрим, как выглядят распределения числовых признаков
def visualize_distributions(titles_values_dict):
  columns = min(3, len(titles_values_dict))
  rows = (len(titles_values_dict) - 1) // columns + 1
  fig = plt.figure(figsize = (columns * 6, rows * 4))
  for i, (title, values) in enumerate(titles_values_dict.items()):
    hist, bins = np.histogram(values, bins = 20)
    ax = fig.add_subplot(rows, columns, i + 1)
    ax.bar(bins[:-1], hist, width = (bins[1] - bins[0]) * 0.7)
    ax.set_title(title)
  plt.show()

visualize_distributions({
    'mileage': train['mileage'].dropna(),
    'modelDate': train['modelDate'].dropna(),
    'productionDate': train['productionDate'].dropna()
})

Распределение признаков ненормальное, поэтому их необходимо нормировать. 

Разбиваем на категориальные признаки и числовые:

In [ ]:
#используем все текстовые признаки как категориальные без предобработки
categorical_features = ['bodyType', 'brand', 'color', 'engineDisplacement', 'enginePower', 'fuelType', 'model_info', 'name',
  'numberOfDoors', 'vehicleTransmission', 'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль']

#используем все числовые признаки
numerical_features = ['mileage', 'modelDate', 'productionDate']

Объединяем тренировочный и тестовый сеты:



In [ ]:
train['sample'] = 1 # помечаем где у нас трейн
test['sample'] = 0 # помечаем где у нас тест
test['price'] = 0 # в тесте у нас нет значения price, мы его должны предсказать, по этому пока просто заполняем нулями

data = test.append(train, sort=False).reset_index(drop=True) # объединяем
print(train.shape, test.shape, data.shape)

Выполняем предобработку данных: удаляем лишнее, заполняем пропуски, если они есть, выполняем нормирование. На CatBoost это не должно повлиять. Наконец, выполняем Label Encoding или One-Hot Encoding.



In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['description','sell_id',], axis = 1, inplace=True)
    
    
    # ################### Numerical Features ############################################################## 
    # Далее заполняем пропуски
    for column in numerical_features:
        df_output[column].fillna(df_output[column].median(), inplace=True)
    # тут ваш код по обработке NAN
    # ....
    
    # Нормализация данных
    scaler = MinMaxScaler()
    for column in numerical_features:
        df_output[column] = scaler.fit_transform(df_output[[column]])[:,0]
    
    
    
    # ################### Categorical Features ############################################################## 
    # Label Encoding
    for column in categorical_features:
        df_output[column] = df_output[column].astype('category').cat.codes
        
    # One-Hot Encoding: в pandas есть готовая функция - get_dummies.
    df_output = pd.get_dummies(df_output, columns=categorical_features, dummy_na=False)
    # убираем признаки которые еще не успели обработать, 
    df_output.drop(['vehicleConfiguration'], axis = 1, inplace=True)
    
    return df_output


Запускаем и смотрим на чистые данные. 



In [ ]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

Мы получили целевую переменную price, дату производства, пробег, столбец с указанием принадлежности к тренировочному или тестовому наборам.   

Затем выделяем тестовую часть:



In [ ]:
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.price.values     # наш таргет
X = train_data.drop(['price'], axis=1)
X_sub = test_data.drop(['price'], axis=1)

Теперь мы можем запустить уже не наивную модель, а построить модель на основе градиентного бустинга при помощи CatBoost. 



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)


In [ ]:
model = CatBoostRegressor(iterations = 5000,
                          #depth=10,
                          #learning_rate = 0.5,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['RMSE', 'MAE'],
                          od_wait=500,
                          #task_type='GPU',
                         )
model.fit(X_train, y_train,
         eval_set=(X_test, y_test),
         verbose_eval=100,
         use_best_model=True,
         #plot=True
         )

Проверяем MAPE у тестовой части: 



In [ ]:
test_predict_catboost = model.predict(X_test)
print(f"TEST mape: {(mape(y_test, test_predict_catboost))*100:0.2f}%")

Мы получили MAPE 13.23%. Это неплохой результат, поскольку CatBoost хорошо работает с небольшим количеством данных и множеством категориальных признаков.

Сохраняем submission: 

In [ ]:
sub_predict_catboost = model.predict(X_sub)
sample_submission['price'] = sub_predict_catboost
sample_submission.to_csv('catboost_submission.csv', index=False)

### 4. Простая нейронная сеть

Перед построением Dense-модели проверим данные и убедимся в том, что их формат нам подходит:



In [ ]:
X_train.head(5)

Полторы тысячи столбцов — это большое количество, но нейронные сети с таким объёмом справляются спокойно. Далее строим обычную Dense-сеть из трёх слоёв с дропаутами:

In [ ]:
model = Sequential()
model.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(256, activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(1, activation="linear"))

In [ ]:
model.summary()

Компилируем модель:



In [ ]:
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

Сохраняем чекпойнты и настраиваем EarlyStopping:



In [ ]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5' , monitor=['val_MAPE'], verbose=0  , mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=50, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

Запускаем обучение на 500 эпох. batch_size устанавливаем достаточно большого размера, поскольку данные занимают немного места.



In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=(X_test, y_test),
                    callbacks=callbacks_list,
                    verbose=0,
                   )

Посмотрим на нашу сеть:



In [ ]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

Кривая обучения получилась достаточно ровной.

Не забываем сохранить:



In [ ]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_1.hdf5')

И смотрим на MAPE:



In [ ]:
test_predict_nn1 = model.predict(X_test)
print(f"TEST mape: {(mape(y_test, test_predict_nn1[:,0]))*100:0.2f}%")

Итак, простая нейронная сеть нам дала 14%.

Сохраняем submission:

In [ ]:
sub_predict_nn1 = model.predict(X_sub)
sample_submission['price'] = sub_predict_nn1[:,0]
sample_submission.to_csv('nn1_submission.csv', index=False)

### КАК МОЖНО УЛУЧШИТЬ ЭТУ СЕТЬ?

1. В нейросеть желательно подавать данные с распределением, близким к нормальному, поэтому от некоторых числовых признаков имеет смысл перед нормализацией взять логарифм. 

Пример: 

In [ ]:
modelDateNorm = np.log(2020 - data['modelDate'])

→ Статья по теме на Хабре (https://habr.com/ru/company/ods/blog/325422/). 

2. Извлечение числовых значений из текста. Парсинг признаков 'engineDisplacement', 'enginePower', 'Владение' для извлечения числовых значений.

3. Cокращение размерности категориальных признаков. Признак 'name' содержит данные, которые уже есть в других столбцах ('enginePower', 'engineDisplacement', 'vehicleTransmission'). Можно удалить эти данные. Затем можно ещё сильнее сократить размерность, например, выделив наличие xDrive в качестве отдельного признака.



### 5. Multi-Input сеть: табличные данные + текст

Вспомните, что вы узнали в модуле об NLP и обработке текста. Итак, у нас есть текстовое описание: 




In [ ]:
data.description

Произведём токенизацию этого поля:



In [ ]:
# TOKENIZER
# The maximum number of words to be used. (most frequent)
MAX_WORDS = 100000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 256

И не забываем разбить данные: 



In [ ]:
# split данных
text_train = data.description.iloc[X_train.index]
text_test = data.description.iloc[X_test.index]
text_sub = data.description.iloc[X_sub.index]

Создаём словарь — в Keras это занимает буквально пару строк:



In [ ]:
%%time
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(data.description)

In [ ]:
tokenize.word_index

Благодаря получившемуся словарю мы сможем произвести дальнейшую векторизацию нашего текста. 



In [ ]:
%%time
text_train_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_train), maxlen=MAX_SEQUENCE_LENGTH)
text_test_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_test), maxlen=MAX_SEQUENCE_LENGTH)
text_sub_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_sub), maxlen=MAX_SEQUENCE_LENGTH)

print(text_train_sequences.shape, text_test_sequences.shape, text_sub_sequences.shape, )

Давайте посмотрим, как теперь выглядит наш текст:

In [ ]:
print(text_train.iloc[6])
print(text_train_sequences[6])

Теперь строим сеть для обработки текста. Для простоты в примере используем LSTM:



In [ ]:
model_nlp = Sequential()
model_nlp.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"))
model_nlp.add(L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,))
model_nlp.add(L.LSTM(256, return_sequences=True))
model_nlp.add(L.Dropout(0.5))
model_nlp.add(L.LSTM(128,))
model_nlp.add(L.Dropout(0.25))
model_nlp.add(L.Dense(64, activation="relu"))
model_nlp.add(L.Dropout(0.25))

Обратите внимание, что эта сеть построена «без головы». Добавим ещё одну сеть — созданную нами ранее для табличных данных:



In [ ]:
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dropout(0.5))

Нам осталось объединить их в Multi-Input сеть, то есть сеть, которая позволяет брать на вход несколько сетей и объединять их результаты. За объединение отвечает слой L.concatenate:

In [ ]:
combinedInput = L.concatenate([model_nlp.output, model_mlp.output])
# being our regression head
head = L.Dense(64, activation="relu")(combinedInput)
head = L.Dense(1, activation="linear")(head)

model = Model(inputs=[model_nlp.input, model_mlp.input], outputs=head)

Кстати, существуют и сети с несколькими выходами, но наша задача на этом этапе — отработать построение сети с несколькими входами и одним выходом.

Смотрим, что у нас получилось:

In [ ]:
model.summary()

Готовимся к обучению: настраиваем показатели, чекпойнты, EarlyStopping:



In [ ]:
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=10, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

Важный момент при обучении — указать отдельно текстовые элементы и табличные, а для валидации не забыть указать несколько входов именно в той последовательности, которую мы установили ранее.



In [ ]:
history = model.fit([text_train_sequences, X_train], y_train,
                    batch_size=512,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=([text_test_sequences, X_test], y_test),
                    callbacks=callbacks_list
                   )

Проверим, как прошло обучение:



In [ ]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

Не забываем сохраняться. Затем смотрим на MAPE:



In [ ]:
test_predict_nn2 = model.predict([text_test_sequences, X_test])
print(f"TEST mape: {(mape(y_test, test_predict_nn2[:,0]))*100:0.2f}%")

Видим, что показатель ошибки стал чуть выше. Скорее всего, это связано с недостаточной обработкой текста перед реализацией модели.

Не забудьте сохранить submission!

### КАК МОЖНО УЛУЧШИТЬ ЭТУ МОДЕЛЬ?

Выделить из описаний часто встречающиеся блоки текста, заменив их на кодовые слова или удалив их.
Сделать предобработку текста, например лемматизацию — алгоритм, который ставит все слова в форму по умолчанию (глаголы в инфинитив и так далее). Это позволит токенайзеру не преобразовывать разные формы слова в разные числа. Статья по теме на Хабре (https://habr.com/ru/company/Voximplant/blog/446738/).
Поработать над алгоритмами очистки и аугментации текста.  

→ Мы закончили работать с NLP, познакомились с Multi-Input сетью — самое время перейти к картинкам!
